<a href="https://colab.research.google.com/github/Mancydeep/Assessment-2-weatherwise-/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyinputplus matplotlib requests

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import os
import matplotlib.pyplot as plt
import pyinputplus as pyip

# Add any other setup code here

## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=5):
    """
    Obtain forecast details for a selected city and duration via wttr.in, limiting the days as recommended.
    """
    endpoint = f"https://wttr.in/{location}?format=j1"
    try:
        result = requests.get(endpoint)
        result.raise_for_status()
        data = result.json()
        data['weather'] = data['weather'][:forecast_days]
        return data
    except Exception as error:
        print(f"Unable to retrieve forecast: {error}")
        return None


## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here

def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Plot daily temperature ranges using a steady, clear area chart.
    """
    day_labels = []
    highs = []
    lows = []

    for section in weather_data['weather']:
        day_labels.append(section['date'])
        highs.append(float(section['maxtempC']))
        lows.append(float(section['mintempC']))

    plt.figure()
    plt.fill_between(day_labels, lows, highs, color='lightgrey', alpha=0.7, label='Temperature Range (°C)')
    plt.plot(day_labels, highs, color='black', label='Max Temp')
    plt.plot(day_labels, lows, color='grey', label='Min Temp')
    plt.title("Temperature Forecast")
    plt.xlabel("Date")
    plt.ylabel("Temp (°C)")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass

## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.